In [ ]:
  !pip install -q pandas chardet openpyxl

import pandas as pd
import chardet, io, csv, unicodedata, re
from google.colab import files as colab_files
from IPython.display import FileLink, display

def normalize(text: str) -> str:
    txt = unicodedata.normalize('NFD', str(text).lower())
    txt = ''.join(ch for ch in txt if unicodedata.category(ch) != 'Mn')
    return re.sub(r'[^a-z0-9 ]', ' ', txt).strip()

def detect_delim(sample: str) -> str:
    try:
        return csv.Sniffer().sniff(sample, delimiters=";,|\t").delimiter
    except:
        return ';'

def read_csv_flexible(content: bytes) -> pd.DataFrame:
    enc = chardet.detect(content)['encoding'] or 'latin1'
    text = content.decode(enc, errors='ignore')
    lines = text.splitlines()
    sample = "\n".join(lines[:10])
    delim = detect_delim(sample)
    header_idx = 0
    for i, line in enumerate(lines):
        nm = normalize(line)
        if ('ventas' in nm and 'margen' in nm) or ('coste' in nm and 'conversiones' in nm):
            header_idx = i
            break
    data = "\n".join(lines[header_idx:])
    return pd.read_csv(io.StringIO(data), sep=delim, engine='python', on_bad_lines='skip')

def find_column(cols, keywords):
    normed = [normalize(c) for c in cols]
    for kw in keywords:
        for orig, nm in zip(cols, normed):
            if kw in nm:
                return orig
    return None

def preprocess_num(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series.astype(str).str.replace(',', '.', regex=False), errors='coerce').fillna(0)


KEYWORDS_BRAND = {
    'brand':       ['marca', 'brand'],
    'product':     ['producto', 'product'],
    'sales':       ['precio venta', 'ventas', 'revenue', 'importe linea', 'imp linea'],
    'margin':      ['margen', 'margin', 'profit'],
    'cost':        ['coste', 'cost', 'gasto', 'spend', 'gasto total'],
    'conversions': ['conversiones', 'convs', 'conversions']
}
EXCLUDE_PATTERN = r'\b(?:ajust|devoluc|cupon|vale)\b'

KEYWORDS_COUNTRY = {
    'line_amt':    ['importe linea', 'imp linea'],
    'unit_price':  ['precio venta', 'precio de venta'],
    'margin':      ['margen', 'margin', 'profit'],
    'cost':        ['coste', 'cost', 'gasto', 'spend', 'gasto total'],
    'conv':        ['conversiones', 'convs', 'conversions'],
    'country':     ['país de venta', 'país', 'pais', 'country', 'region'],
    'product':     ['producto', 'product']
}
EXCLUDE_PROD_PATTERN = r'\b(?:ajust|devoluc|cupon|vale)\b'

print("Sube uno o varios CSVs de ventas/gastos:")
uploaded = colab_files.upload()



ventas_list, gastos_list = [], []
all_brands = set()

for fname, content in uploaded.items():
    df = read_csv_flexible(content)
    df.columns = df.columns.str.strip()

    c_brand  = find_column(df.columns, KEYWORDS_BRAND['brand'])
    c_prod   = find_column(df.columns, KEYWORDS_BRAND['product'])
    c_sales  = find_column(df.columns, KEYWORDS_BRAND['sales'])
    c_margin = find_column(df.columns, KEYWORDS_BRAND['margin'])
    c_cost   = find_column(df.columns, KEYWORDS_BRAND['cost'])
    c_conv   = find_column(df.columns, KEYWORDS_BRAND['conversions'])

    marcas_orig = df[c_brand].fillna('').astype(str) if c_brand else pd.Series(['']*len(df))
    marcas_norm = marcas_orig.apply(normalize)
    all_brands.update(marcas_norm.unique())

    if c_sales and c_margin:
        sub = pd.DataFrame({
            'Marca_original': marcas_orig,
            'Marca_norm':     marcas_norm,
            'Ventas':         preprocess_num(df[c_sales]),
            'Margen':         preprocess_num(df[c_margin])
        })
        if c_prod:
            excl = df[c_prod].astype(str).str.contains(EXCLUDE_PATTERN, na=False)
            sub = sub[~excl]
        ventas_list.append(sub[['Marca_norm', 'Ventas', 'Margen']])

    if c_cost and c_conv:
        sub2 = pd.DataFrame({
            'Marca_norm':   marcas_norm,
            'Coste':        preprocess_num(df[c_cost]),
            'Conversiones': preprocess_num(df[c_conv])
        })
        gastos_list.append(sub2)

df_vb = pd.concat(ventas_list, ignore_index=True).groupby('Marca_norm', as_index=False).sum() \
       if ventas_list else pd.DataFrame(columns=['Marca_norm', 'Ventas', 'Margen'])
df_gb = pd.concat(gastos_list, ignore_index=True).groupby('Marca_norm', as_index=False).sum() \
       if gastos_list else pd.DataFrame(columns=['Marca_norm', 'Coste', 'Conversiones'])

brands_df = pd.DataFrame({'Marca_norm': sorted(all_brands)})
res_brand = (brands_df
               .merge(df_vb, on='Marca_norm', how='left')
               .merge(df_gb, on='Marca_norm', how='left')
               .fillna(0))
res_brand['Marca'] = res_brand['Marca_norm'].str.title().replace({'': ''})
res_brand = res_brand[['Marca', 'Ventas', 'Margen', 'Coste', 'Conversiones']]
res_brand = res_brand.sort_values('Ventas', ascending=False).reset_index(drop=True)

tot_b = res_brand[['Ventas', 'Margen', 'Coste', 'Conversiones']].sum()
res_brand = pd.concat([
    res_brand,
    pd.DataFrame([{'Marca': 'Total', **tot_b.to_dict()}])
], ignore_index=True)

print("###  RESUMEN POR MARCA ###")
display(res_brand)
res_brand.to_excel('resumen_corregido_marcas.xlsx', index=False)
colab_files.download('resumen_corregido_marcas.xlsx')
display(FileLink('resumen_corregido_marcas.xlsx'))


ventas_ctry, gastos_ctry = [], []
all_countries = set()

for fname, content in uploaded.items():
    enc = chardet.detect(content)['encoding'] or 'latin1'
    raw = content.decode(enc, errors='ignore')
    delim = detect_delim(raw[:2000])
    df = pd.read_csv(io.StringIO(raw), sep=delim, engine='python', on_bad_lines='skip')
    df.columns = df.columns.str.strip()

    c_ctry  = find_column(df.columns, KEYWORDS_COUNTRY['country'])
    c_prod  = find_column(df.columns, KEYWORDS_COUNTRY['product'])
    c_sales = find_column(df.columns, KEYWORDS_COUNTRY['line_amt']) or \
              find_column(df.columns, KEYWORDS_COUNTRY['unit_price'])
    c_marg  = find_column(df.columns, KEYWORDS_COUNTRY['margin'])
    c_cost  = find_column(df.columns, KEYWORDS_COUNTRY['cost'])
    c_conv  = find_column(df.columns, KEYWORDS_COUNTRY['conv'])

    if not c_ctry:
        continue

    paises_orig = df[c_ctry].fillna('').astype(str)
    paises_norm = paises_orig.apply(normalize)
    all_countries.update(paises_norm.unique())

    if c_sales and c_marg:
        subv = pd.DataFrame({
            'País_original': paises_orig,
            'País_norm':     paises_norm,
            'Ventas':        preprocess_num(df[c_sales]),
            'Margen':        preprocess_num(df[c_marg])
        })
        if c_prod:
            excl = df[c_prod].astype(str).str.contains(EXCLUDE_PROD_PATTERN, na=False)
            subv = subv[~excl]
        ventas_ctry.append(subv[['País_norm', 'Ventas', 'Margen']])

    if c_cost and c_conv:
        subg = pd.DataFrame({
            'País_norm':     paises_norm,
            'Coste':         preprocess_num(df[c_cost]),
            'Conversiones':  preprocess_num(df[c_conv])
        })
        gastos_ctry.append(subg)

df_vp = pd.concat(ventas_ctry, ignore_index=True).groupby('País_norm', as_index=False).sum() \
       if ventas_ctry else pd.DataFrame(columns=['País_norm', 'Ventas', 'Margen'])
df_gp = pd.concat(gastos_ctry, ignore_index=True).groupby('País_norm', as_index=False).sum() \
       if gastos_ctry else pd.DataFrame(columns=['País_norm', 'Coste', 'Conversiones'])

countries_df = pd.DataFrame({'País_norm': sorted(all_countries)})
res_country = (countries_df
                 .merge(df_vp, on='País_norm', how='left')
                 .merge(df_gp, on='País_norm', how='left')
                 .fillna(0))
res_country['País'] = res_country['País_norm'].str.title().replace({'': ''})
res_country = res_country[['País', 'Ventas', 'Margen', 'Coste', 'Conversiones']]
res_country = res_country.sort_values('Ventas', ascending=False).reset_index(drop=True)

tot_ctry = res_country[['Ventas', 'Margen', 'Coste', 'Conversiones']].sum()
res_country = pd.concat([
    res_country,
    pd.DataFrame([{'País': 'Total', **tot_ctry.to_dict()}])
], ignore_index=True)

print("RESUMEN POR PAÍS")
display(res_country)
res_country.to_excel('resumen_corregido_paises.xlsx', index=False)
colab_files.download('resumen_corregido_paises.xlsx')
display(FileLink('resumen_corregido_paises.xlsx'))




---

